# Linear Regression with Multiple Variable
In this part, you will implement linear regression with multiple variables to predict the prices of houses. Suppose you are selling your house and you want to know what a good market price would be. One way to do this is to first collect information on recent houses sold and make a model of housing prices.<br>

The file <strong><em>ex1data2.csv</em></strong> contains a training set of housing prices in Portland, Oregon.<br>
The structure of the dataset described blow:<br>
1. First column = <strong>size of the house (in square feet)</strong> 
2. Second column = <strong>number of bedrooms</strong>
3. Third column = <strong>price of the house</strong>

A negative value for profit indicates a loss.<br>
<br> <br>
<strong>
Our assignment has these sections:
1. Plotting the Data
    1. Loading dataset
    2. Ploting scatter
2. Feature Normalization
3. Gradient Descent
    1. Update Equations
    2. Implementation
    3. Computing the cost J(θ)
    4. Selecting Learning Rates (alpha)
    5. Gradient descent
    6. Visualization of Fitted Model
4. Normal Equations
5. Visualization J(θ)
    1. Surface
    2. Contour
</strong>

In each section full description provided.